In [1]:
# Import the dependencies.
import pandas as pd # read our CSV file and create the locations and measurements from the DataFrame
import gmaps # create heatmaps and the locations map
import requests # request to the Google Places JSON file
# Import the API key.
from config import g_key # create heatmaps and the locations map

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

City_ID        City Country                 Date      Lat       Lng  \
0        0    Hermanus      ZA  2021-11-10 04:07:13 -34.4187   19.2345   
1        1   Geraldton      AU  2021-11-10 04:07:14 -28.7667  114.6000   
2        2  Georgetown      MY  2021-11-10 04:00:40   5.4112  100.3354   
3        3   Busselton      AU  2021-11-10 04:07:14 -33.6500  115.3333   
4        4    Victoria      HK  2021-11-10 04:00:27  22.2855  114.1577   

   Max Temp  Humidity  Cloudiness  Wind Speed  
0     14.39        77          44        1.50  
1     23.23        31           0        7.72  
2     27.96        85          20        0.45  
3     18.50        43           0        3.47  
4     24.58        33          33        1.34

The data we use for any mapping must be either an integer or a floating-point decimal number

## Create Heatmaps

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=[max(temp, 0) for temp in max_temp],
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights = clouds,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights = wind,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Filter Cities by Preferred Temperature

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 15
What is the maximum temperature you would like for your trip? 23


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

City_ID            City Country                 Date      Lat       Lng  \
3         3       Busselton      AU  2021-11-10 04:07:14 -33.6500  115.3333   
5         5           Avera      US  2021-11-10 04:07:15  33.1940  -82.5271   
7         7      Sao Mateus      BR  2021-11-10 04:07:16 -18.7161  -39.8589   
15       15         Mataura      NZ  2021-11-10 04:07:19 -46.1927  168.8643   
19       19    Myrtle Beach      US  2021-11-10 04:04:52  33.6891  -78.8867   
22       22  Ribeira Grande      PT  2021-11-10 04:07:23  38.5167  -28.7000   
23       23          Hobart      AU  2021-11-10 04:05:33 -42.8794  147.3294   
31       31           Kidal      ML  2021-11-10 04:07:26  18.4411    1.4078   
32       32        Cidreira      BR  2021-11-10 04:07:27 -30.1811  -50.2056   
36       36         Phaltan      IN  2021-11-10 04:07:28  17.9833   74.4333   

    Max Temp  Humidity  Cloudiness  Wind Speed  
3      18.50        43           0        3.47  
5      15.93        42          90        2.22  
7      22.45        94          98        0.94  
15     15.00        93         100        2.16  
19     15.03        74           1        0.00  
22     19.50        79          32        5.80  
23     15.06        90          75        0.89  
31     20.63        12          16        4.45  
32     17.10        77           7        2.40  
36     22.74        42         100        2.62

In [10]:
preferred_cities_df.count()

City_ID       143
City          143
Country       140
Date          143
Lat           143
Lng           143
Max Temp      143
Humidity      143
Cloudiness    143
Wind Speed    143
dtype: int64

In [11]:
preferred_cities_df = preferred_cities_df.dropna()

## Retrieve Hotels from a Nearby Search

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

City Country  Max Temp      Lat       Lng Hotel Name
3        Busselton      AU     18.50 -33.6500  115.3333           
5            Avera      US     15.93  33.1940  -82.5271           
7       Sao Mateus      BR     22.45 -18.7161  -39.8589           
15         Mataura      NZ     15.00 -46.1927  168.8643           
19    Myrtle Beach      US     15.03  33.6891  -78.8867           
22  Ribeira Grande      PT     19.50  38.5167  -28.7000           
23          Hobart      AU     15.06 -42.8794  147.3294           
31           Kidal      ML     20.63  18.4411    1.4078           
32        Cidreira      BR     17.10 -30.1811  -50.2056           
36         Phaltan      IN     22.74  17.9833   74.4333

In [13]:
# URL and Parameters

# Set the base Nearby URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found in {row['City']}... skipping.")

Hotel not found in Avera... skipping.
Hotel not found in Kidal... skipping.
Hotel not found in Chunhuhub... skipping.
Hotel not found in Tessalit... skipping.
Hotel not found in Boda... skipping.
Hotel not found in Caconda... skipping.
Hotel not found in Libenge... skipping.
Hotel not found in Goundam... skipping.
Hotel not found in Amapa... skipping.
Hotel not found in Kibara... skipping.
Hotel not found in Adrar... skipping.
Hotel not found in Maridi... skipping.
Hotel not found in Tambopata... skipping.
Hotel not found in Taoudenni... skipping.
Hotel not found in Awjilah... skipping.
Hotel not found in Nioro... skipping.
Hotel not found in Abu Samrah... skipping.


## Map Vacation Criteria

In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))